# Step 08: Control Totals

### Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.sqlserver import execute_query_from_file
from helpers.database import execute_query
from helpers.configuration import read_configuration
from helpers.irp_integration import IRPClient
from helpers.control_totals import validate_geohaz_thresholds, get_import_file_mapping_from_config

# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_08_Control_Totals.ipynb')

# Display context
ux.header("Control Totals Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

### Verify Configuration

In [ ]:
# Verify configuration exists and is valid
ux.header("Configuration Verification")

# Get cycle ID
cycle_result = execute_query(
    "SELECT id FROM irp_cycle WHERE cycle_name = %s",
    (context.cycle_name,)
)

if cycle_result.empty:
    raise ValueError(f"Cycle not found: {context.cycle_name}")

cycle_id = int(cycle_result.iloc[0]['id'])  # Convert numpy.int64 to Python int

# Get configuration for this cycle
config_result = execute_query(
    "SELECT id, status, created_ts FROM irp_configuration WHERE cycle_id = %s ORDER BY created_ts DESC LIMIT 1",
    (cycle_id,)
)

if config_result.empty:
    ux.error("✗ No configuration found for this cycle")
    ux.info("Please complete Step 02: Validate Configuration File first")
    raise ValueError("No configuration found for cycle")

config_id = int(config_result.iloc[0]['id'])  # Convert numpy.int64 to Python int
config_status = config_result.iloc[0]['status']
config_created = config_result.iloc[0]['created_ts']

# Verify status is VALID or ACTIVE
if config_status not in ['VALID', 'ACTIVE']:
    ux.error(f"✗ Configuration status is '{config_status}' (expected VALID or ACTIVE)")
    raise ValueError(f"Configuration must be VALID or ACTIVE, found: {config_status}")

# Display configuration summary
config_info = [
    ["Configuration ID", config_id],
    ["Status", config_status],
    ["Created", config_created.strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(config_info, headers=["Property", "Value"])
ux.success("✓ Configuration verified")

# Read configuration data
config_data = read_configuration(config_id)

# Extract configuration_data JSONB field
configuration_data = config_data.get('configuration_data', {})
metadata = configuration_data.get('Metadata', {})

step.log(f"Configuration verified: ID={config_id}, Status={config_status}")

### Validate Workspace EDM

In [ ]:
# Get Workspace EDM
irp_client = IRPClient()
workspace_edm = "WORKSPACE_EDM"
workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
if (len(workspace_edms) == 0):
    moody_job_id = irp_client.edm.submit_create_edm_job(edm_name=workspace_edm)
    irp_client.job.poll_risk_data_job_to_completion(moody_job_id)
    workspace_edms = irp_client.edm.search_edms(filter=f"exposureName=\"{workspace_edm}\"")
workspace_edm = workspace_edms[0]
workspace_edm_full_name = workspace_edm['databaseName']
print(f'Workspace EDM Full Name: {workspace_edm_full_name}')

### Exposure Control Totals

In [ ]:
sql_script_name = '3d_RMS_EDM_CONTROL_Totals.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": metadata.get('Cycle Type'),
    "DATE_VALUE": metadata.get('Current Date Value')
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = 'DATABRIDGE'
)

### GeoHaz Control Totals

In [ ]:
# Execute GeoHaz Control Totals Query
sql_script_name = '3e_GeocodingSummary.sql'
params = {
    "WORKSPACE_EDM": workspace_edm_full_name,
    "CYCLE_TYPE": metadata.get('Cycle Type'),
    "DATE_VALUE": metadata.get('Current Date Value')
}

result = execute_query_from_file(
    file_path = f'control_totals/{sql_script_name}',
    params = params,
    connection = 'DATABRIDGE'
)

# Extract geocoding results (first result set)
geocoding_results = result[0] if isinstance(result, list) else result

ux.success(f"✓ GeoHaz query executed: {len(geocoding_results)} geocoding records retrieved")

### Validate GeoHaz Thresholds

In [ ]:
# Validate geocoding results against configuration thresholds
ux.header("GeoHaz Threshold Validation")

# Get GeoHaz thresholds from configuration
geohaz_thresholds = configuration_data.get('GeoHaz Thresholds', [])

if not geohaz_thresholds:
    ux.warning("⚠ No GeoHaz thresholds found in configuration - skipping validation")
else:
    # Get portfolio to import file mapping from configuration
    import_file_mapping = get_import_file_mapping_from_config(configuration_data)
    
    # Perform validation
    validation_results, all_passed = validate_geohaz_thresholds(
        geocoding_results=geocoding_results,
        geohaz_thresholds=geohaz_thresholds,
        import_file_mapping=import_file_mapping
    )
    
    # Handle empty validation results (e.g., no matching portfolios or all portfolios empty)
    if validation_results.empty:
        ux.warning("⚠ No geocoding results matched the configured thresholds")
        ux.info("This may occur when:")
        ux.info("  - No portfolios in geocoding results match Import File names in thresholds")
        ux.info("  - All matching portfolios had zero locations")
        step.log("GeoHaz validation: No matching results to validate")
    else:
        # Display summary
        total_checks = len(validation_results)
        passed_checks = len(validation_results[validation_results['Status'] == 'PASS'])
        failed_checks = len(validation_results[validation_results['Status'] == 'FAIL'])
        
        summary_data = [
            ["Total Checks", total_checks],
            ["Passed", passed_checks],
            ["Failed", failed_checks]
        ]
        ux.table(summary_data, headers=["Metric", "Count"])
        
        if all_passed:
            ux.success("✓ All geocoding thresholds met!")
        else:
            ux.error(f"✗ {failed_checks} geocoding threshold(s) not met")
        
        # Display detailed results
        ux.subheader("Detailed Validation Results")
        
        # Show failures first
        failures = validation_results[validation_results['Status'] == 'FAIL']
        if not failures.empty:
            ux.warning(f"Threshold Violations ({len(failures)}):")
            print(failures.to_string(index=False))
            print()
        
        # Show passes
        passes = validation_results[validation_results['Status'] == 'PASS']
        if not passes.empty:
            ux.info(f"Passed Thresholds ({len(passes)}):")
            print(passes.to_string(index=False))
        
        # Log validation status
        if all_passed:
            step.log(f"GeoHaz validation: All {total_checks} thresholds met")
        else:
            step.log(f"GeoHaz validation: {failed_checks}/{total_checks} thresholds failed")

### Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data with control totals summary
output_data = {
    'exposure_control_totals_executed': True,
    'geohaz_control_totals_executed': True
}

# Add GeoHaz validation results if available and non-empty
if 'validation_results' in locals() and not validation_results.empty:
    total_checks = len(validation_results)
    passed_checks = len(validation_results[validation_results['Status'] == 'PASS'])
    failed_checks = len(validation_results[validation_results['Status'] == 'FAIL'])
    
    output_data['geohaz_validation'] = {
        'total_checks': int(total_checks),
        'passed_checks': int(passed_checks),
        'failed_checks': int(failed_checks),
        'all_passed': bool(all_passed)
    }

# Complete the step
step.complete(output_data)

# Send Teams notification for milestone completion
import os
from helpers.teams_notification import TeamsNotificationClient
from helpers.database import get_current_schema
teams = TeamsNotificationClient()

# Build action buttons with notebook link and dashboard
actions = []
base_url = os.environ.get('TEAMS_DEFAULT_JUPYTERLAB_URL', '')
if base_url:
    notebook_path = str(context.notebook_path)
    if 'workflows' in notebook_path:
        rel_path = notebook_path.split('workflows')[-1].lstrip('/\\')
        notebook_url = f"{base_url.rstrip('/')}/lab/tree/workspace/workflows/{rel_path}"
        actions.append({"title": "Open Notebook", "url": notebook_url})

dashboard_url = os.environ.get('TEAMS_DEFAULT_DASHBOARD_URL', '')
if dashboard_url:
    schema = get_current_schema()
    cycle_dashboard_url = f"{dashboard_url.rstrip('/')}/{schema}/cycle/{context.cycle_name}"
    actions.append({"title": "View Cycle Dashboard", "url": cycle_dashboard_url})

# Build summary message
summary_parts = ["**Control Totals validation completed.**"]
if 'validation_results' in locals() and not validation_results.empty:
    summary_parts.append(f"GeoHaz Validation: {passed_checks}/{total_checks} checks passed")
    if not all_passed:
        summary_parts.append(f"⚠ {failed_checks} threshold(s) not met")

teams.send_success(
    title=f"[{context.cycle_name}] Control Totals Completed",
    message=f"**Cycle:** {context.cycle_name}\n"
            f"**Stage:** {context.stage_name}\n"
            f"**Step:** {context.step_name}\n\n" +
            "\n".join(summary_parts),
    actions=actions if actions else None
)

ux.success("\n" + "="*60)
ux.success("✓ CONTROL TOTALS VALIDATION COMPLETED")
ux.success("="*60)

# Display summary
if 'validation_results' in locals() and not validation_results.empty:
    ux.info(f"\nGeoHaz Validation: {passed_checks}/{total_checks} checks passed")
    if all_passed:
        ux.success("  ✓ All geocoding thresholds met")
    else:
        ux.warning(f"  ⚠ {failed_checks} threshold(s) not met")
elif 'validation_results' in locals() and validation_results.empty:
    ux.info("\nGeoHaz validation: No matching results to validate")
else:
    ux.info("\nGeoHaz validation: Skipped (no thresholds configured)")

ux.info("\nNext: Proceed to Stage 04 (Analysis Execution)")